In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
import numpy as np

In [2]:
BATCH_SIZE = 128
EPOCHS = 50
LAYER_SIZE = 30
S_LAYER_SIZE = 15
LATENT_SIZE = 4

In [3]:
# Load the data from the file

saved = np.load("testing2.npy", allow_pickle=True)

# Make a trainging and testing batch
train_data = torch.Tensor(saved[:int(len(saved)*0.5)])
test_data = torch.Tensor(saved[int(len(saved)*0.5):])
print(train_data.shape)
INPUT_SIZE = len(test_data[0])
INPUT_SIZE

torch.Size([50000, 12])


12

In [4]:
device = "cuda"

In [5]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        # Encoder
        self.l1 = nn.Linear(INPUT_SIZE, LAYER_SIZE)
        self.ltest1 = nn.Linear(LAYER_SIZE, S_LAYER_SIZE)
        self.l2a = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        self.l2b = nn.Linear(S_LAYER_SIZE, LATENT_SIZE)
        
        # Decoder
        self.l3 = nn.Linear(LATENT_SIZE, S_LAYER_SIZE)
        self.ltest2 = nn.Linear(S_LAYER_SIZE, LAYER_SIZE)
        self.l4 = nn.Linear(LAYER_SIZE, INPUT_SIZE)
            
    # Run some data through the encoder
    def encode(self, x):
        out = F.relu(self.l1(x))
        out = F.relu(self.ltest1(out))
        # return the mu and the sigma
        return self.l2a(out), self.l2b(out)
    
    # The reparameterization trick, taken from https://github.com/pytorch/examples/blob/master/vae/main.py
    def reparameterize(self, mu, sigma):
        std = torch.exp(0.5*sigma)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, x):
        out = F.relu(self.l3(x))
        out = F.relu(self.ltest2(out))
        return torch.sigmoid(self.l4(out)) # sigmoid vs tanh
    
    def forward(self, x):
        # TODO: check the shape of x to be sure we have the right input
        mu, sigma = self.encode(x)
        z = self.reparameterize(mu, sigma)
        # The loss function needs the mu and the sigma so just return them here
        return self.decode(z), mu, sigma

In [6]:
# Reconstruction + KL divergence losses summed over all elements and batch
# Taken from https://github.com/pytorch/examples/blob/master/vae/main.py
def loss_function(recon_x, x, mu, sigma):
    BCE = F.binary_cross_entropy(recon_x, x, reduction='sum')
    KLD = -0.5 * torch.sum(1 + sigma - mu.pow(2) - sigma.exp())
    return BCE + KLD

In [7]:
def train(e_count, model, optimizer, name, data):
    model.train()
    train_loss = 0
    for i in range(0, len(data), BATCH_SIZE):
        batch = data[i:i+BATCH_SIZE].to(device)
        model.zero_grad()
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        
    print(f"Model: {name}, -- Epoch: {e_count}, Loss: {train_loss/len(data)}")        
        
    
    

In [8]:
# Lets use CPU
def test(e_count, model, optimizer):
    model.eval()
    test_loss = 0
    for i in range(0, len(test_data), BATCH_SIZE):
        batch = test_data[i:i+BATCH_SIZE].to(device)
        recons, mu, sigma = model(batch)
        loss = loss_function(recons, batch, mu, sigma)
        test_loss += loss.item()
    print(f"TEST Epoch: {e_count}, Loss: {test_loss/len(test_data)}") 
        
        

In [9]:
def get_data(data, balanced, size):
    res = []
    for d in data:
        if d[10] == int(balanced):
            res.append(d)
        if len(res) == size:
            break
    return torch.Tensor(res)

In [10]:
train_data=train_data[torch.randperm(train_data.size()[0])]


balanced = torch.cat((get_data(saved[:int(len(saved)*0.5)], True, 5000), get_data(saved[:int(len(saved)*0.5)], False, 5000)), 0)

In [11]:
len(balanced)

10000

In [12]:
balanced

tensor([[0.5003, 0.4994, 0.5108,  ..., 0.0500, 1.0000, 1.0000],
        [0.4998, 0.4998, 0.5004,  ..., 0.0500, 1.0000, 1.0000],
        [0.5003, 0.5005, 0.5002,  ..., 0.0500, 1.0000, 1.0000],
        ...,
        [0.5003, 0.5001, 0.5003,  ..., 0.0500, 0.0000, 0.0000],
        [0.4997, 0.5004, 0.5003,  ..., 0.0500, 0.0000, 0.0000],
        [0.4997, 0.5004, 0.5003,  ..., 0.0500, 0.0000, 0.0000]])

In [13]:
balanced=balanced[torch.randperm(balanced.size()[0])]
# SHUFFLE

In [14]:
balanced

tensor([[0.4997, 0.4999, 0.4998,  ..., 0.0500, 0.0000, 0.0000],
        [0.4997, 0.5001, 0.4999,  ..., 0.0500, 0.0000, 0.0000],
        [0.4997, 0.4999, 0.5000,  ..., 0.0500, 1.0000, 1.0000],
        ...,
        [0.4995, 0.5001, 0.5002,  ..., 0.0500, 1.0000, 1.0000],
        [0.5005, 0.4996, 0.5003,  ..., 0.0500, 0.0000, 0.0000],
        [0.4996, 0.5003, 0.5003,  ..., 0.0500, 1.0000, 1.0000]])

In [15]:
# TWO DATA SETS, TWO MODELS
train_data = train_data[:len(balanced)]

vae1 = VAE().to(device)
opt1 = optim.Adam(vae1.parameters(), lr=0.001)

vae2 = VAE().to(device)
opt2 = optim.Adam(vae2.parameters(), lr=0.001)

In [16]:
for i in range(EPOCHS):
    train(i, vae1,opt1, "UNBALANCED" ,train_data)

Model: UNBALANCED, -- Epoch: 0, Loss: 8.139811200714112
Model: UNBALANCED, -- Epoch: 1, Loss: 7.430460530853272
Model: UNBALANCED, -- Epoch: 2, Loss: 7.342006398773194
Model: UNBALANCED, -- Epoch: 3, Loss: 7.319198638153076
Model: UNBALANCED, -- Epoch: 4, Loss: 7.277420015716553
Model: UNBALANCED, -- Epoch: 5, Loss: 7.229843869018555
Model: UNBALANCED, -- Epoch: 6, Loss: 7.190247055053711
Model: UNBALANCED, -- Epoch: 7, Loss: 7.176218574523926
Model: UNBALANCED, -- Epoch: 8, Loss: 7.166717330169678
Model: UNBALANCED, -- Epoch: 9, Loss: 7.143388742065429
Model: UNBALANCED, -- Epoch: 10, Loss: 7.150472478485107
Model: UNBALANCED, -- Epoch: 11, Loss: 7.13665004196167
Model: UNBALANCED, -- Epoch: 12, Loss: 7.1203114151000975
Model: UNBALANCED, -- Epoch: 13, Loss: 7.125505443572998
Model: UNBALANCED, -- Epoch: 14, Loss: 7.091716706848144
Model: UNBALANCED, -- Epoch: 15, Loss: 7.076195491027832
Model: UNBALANCED, -- Epoch: 16, Loss: 7.077371883392334
Model: UNBALANCED, -- Epoch: 17, Loss: 7.

In [17]:
for i in range(EPOCHS):
    train(i, vae2,opt2, "BALANCED" ,balanced)

Model: BALANCED, -- Epoch: 0, Loss: 8.181881665802003
Model: BALANCED, -- Epoch: 1, Loss: 7.8778060455322265
Model: BALANCED, -- Epoch: 2, Loss: 7.83056778717041
Model: BALANCED, -- Epoch: 3, Loss: 7.826215831756592
Model: BALANCED, -- Epoch: 4, Loss: 7.80645315322876
Model: BALANCED, -- Epoch: 5, Loss: 7.599022688293457
Model: BALANCED, -- Epoch: 6, Loss: 7.40401178894043
Model: BALANCED, -- Epoch: 7, Loss: 7.367986540222168
Model: BALANCED, -- Epoch: 8, Loss: 7.343864087677002
Model: BALANCED, -- Epoch: 9, Loss: 7.352129936218262
Model: BALANCED, -- Epoch: 10, Loss: 7.352031071472168
Model: BALANCED, -- Epoch: 11, Loss: 7.362678592681885
Model: BALANCED, -- Epoch: 12, Loss: 7.3516174438476565
Model: BALANCED, -- Epoch: 13, Loss: 7.35466658782959
Model: BALANCED, -- Epoch: 14, Loss: 7.3446615715026855
Model: BALANCED, -- Epoch: 15, Loss: 7.3532524589538575
Model: BALANCED, -- Epoch: 16, Loss: 7.3638721961975095
Model: BALANCED, -- Epoch: 17, Loss: 7.356624504089355
Model: BALANCED, --

In [18]:
# Shuffle test data
test_data=test_data[torch.randperm(test_data.size()[0])]

ex_done = None
ex_notdone = None
for a in test_data:
    if a[10] == 1:
        ex_done = a
        break

for a in test_data:
    if a[10] == 0:
        ex_notdone = a
        break

In [19]:
ex_done[10]

tensor(1.)

In [20]:
list(vae1(ex_done.to("cuda")))[0][10]

tensor(0.9420, device='cuda:0', grad_fn=<SelectBackward>)

In [21]:
list(vae2(ex_done.to("cuda")))[0][10]

tensor(0.9915, device='cuda:0', grad_fn=<SelectBackward>)

In [22]:
ex_notdone[10]

tensor(0.)

In [23]:
list(vae1(ex_notdone.to("cuda")))[0][10]

tensor(0.4744, device='cuda:0', grad_fn=<SelectBackward>)

In [24]:
list(vae2(ex_notdone.to("cuda")))[0][10]

tensor(3.7408e-05, device='cuda:0', grad_fn=<SelectBackward>)

In [25]:
count = 0
for d in balanced:
    if d[10] == 1:
        count += 1
print((count/len(train_data))*100)

50.0


In [26]:
count = 0
for d in train_data:
    if d[10] == 1:
        count += 1
print((count/len(train_data))*100)

17.37


In [27]:
list(vae2(ex_done.to("cuda")))[0][10]

tensor(0.9949, device='cuda:0', grad_fn=<SelectBackward>)

In [28]:
list(vae2(ex_notdone.to("cuda")))[0][10]

tensor(0.1503, device='cuda:0', grad_fn=<SelectBackward>)

In [29]:
from torch.autograd import Variable

In [30]:
sample = Variable(torch.randn(1, LATENT_SIZE))

In [31]:
res = vae2.decode(sample.to("cuda"))

In [32]:
res

tensor([[0.5003, 0.4995, 0.5001, 0.4995, 0.5190, 0.4988, 0.5015, 0.5031, 0.4994,
         0.0501, 0.9899, 0.9901]], device='cuda:0', grad_fn=<SigmoidBackward>)

In [35]:
res = [0, 0, 0]
k = []
for i in range(10000):
    sample = Variable(torch.randn(1, LATENT_SIZE))
    a = vae1.decode(sample.to("cuda"))[0]
    num = float(sum(list(a[10:])))
    k.append(num)
    if num < 0.2:
        res[0] += 1
    elif num <0.7:
        res[1] += 1
    else:
        res[2] += 1
        
d = 0
nd = 0

for a in test_data:
    if a[10] == 1:
        d += 1
    else:
        nd += 1
        
        
print(f"UNBALANCED DATASET")
print(f"counts of intervals: [0, 0.1]: {res[0]}, [0.1, 0.7]: {res[1]}, [0.7, 1]: {res[2]}")
print(f"-- so the ratio to done and not done (deleting the middle ones) is {res[2]/(res[2]+res[0])}")
print(f"The real ration is {d/(nd+d)}")

UNBALANCED DATASET
counts of intervals: [0, 0.1]: 7958, [0.1, 0.7]: 559, [0.7, 1]: 1483
-- so the ratio to done and not done (deleting the middle ones) is 0.15708081770998836
The real ration is 0.17494
